In [1]:
import pandas as pd 
import numpy as np 
import torch
import torch.nn as nn

In [2]:
data = pd.read_csv('mercedes_benz_sales_2020_2025.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12132666 entries, 0 to 12132665
Data columns (total 9 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   Model             object
 1   Year              int64 
 2   Region            object
 3   Color             object
 4   Fuel Type         object
 5   Base Price (USD)  int64 
 6   Horsepower        int64 
 7   Sales Volume      int64 
 8   Turbo             object
dtypes: int64(4), object(5)
memory usage: 833.1+ MB


In [3]:
data.head(10)

,Model,Year,Region,Color,Fuel Type,Base Price (USD),Horsepower,Sales Volume,Turbo
0,A-Class,2020,Global,Yellow,Diesel,41265,252,1,Yes
1,A-Class,2020,Global,Black,Petrol,51023,249,1,No
2,A-Class,2020,Global,Grey,Petrol,72819,341,1,Yes
3,A-Class,2020,Global,Black,Petrol,62480,385,1,Yes
4,A-Class,2020,Global,White,Petrol,35189,337,1,Yes
5,A-Class,2020,Global,Black,Petrol,66551,393,1,Yes
6,A-Class,2020,Global,White,Diesel,56777,316,1,Yes
7,A-Class,2020,Global,Yellow,Diesel,58897,414,1,Yes
8,A-Class,2020,Global,Black,Diesel,79974,413,1,Yes
9,A-Class,2020,Global,Black,Petrol,36021,151,1,Yes


In [4]:
data.isnull().sum()

Model               0
Year                0
Region              0
Color               0
Fuel Type           0
Base Price (USD)    0
Horsepower          0
Sales Volume        0
Turbo               0
dtype: int64

In [5]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
data['enc_model'] = enc.fit_transform(data['Model'])
data['enc_region'] = enc.fit_transform(data['Region'])
data['enc_color'] = enc.fit_transform(data['Color'])
data['enc_turbo'] = enc.fit_transform(data['Turbo'])
data['enc_fuel'] =enc.fit_transform(data['Fuel Type'])

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12132666 entries, 0 to 12132665
Data columns (total 14 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   Model             object
 1   Year              int64 
 2   Region            object
 3   Color             object
 4   Fuel Type         object
 5   Base Price (USD)  int64 
 6   Horsepower        int64 
 7   Sales Volume      int64 
 8   Turbo             object
 9   enc_model         int32 
 10  enc_region        int32 
 11  enc_color         int32 
 12  enc_turbo         int32 
 13  enc_fuel          int32 
dtypes: int32(5), int64(4), object(5)
memory usage: 1.0+ GB


In [6]:
x = data.drop(data[['Model','Region','Color','Fuel Type','Turbo','Base Price (USD)']],axis=1)
y = data['Base Price (USD)']

In [7]:
from sklearn.feature_selection import SelectKBest,f_regression

enc = SelectKBest(f_regression,k=6)
x_new = enc.fit_transform(x,y)
selector = x.columns[enc.get_support()]
print(selector)
x_new = x[selector]

Index(['Year', 'Horsepower', 'enc_model', 'enc_color', 'enc_turbo',
       'enc_fuel'],
      dtype='object')


In [8]:
from sklearn.preprocessing import StandardScaler

enc = StandardScaler()

x_new = enc.fit_transform(x_new)

In [9]:
y = (y - y.mean())/np.std(y)

In [9]:
y

0          -0.928031
1          -0.784179
2          -0.462865
3          -0.615281
4          -1.017602
              ...   
12132661    1.701658
12132662    2.173178
12132663    2.310499
12132664    3.048817
12132665    4.117398
Name: Base Price (USD), Length: 12132666, dtype: float64

In [10]:
x_new = torch.tensor(x_new,dtype=torch.float)
y = torch.tensor(y.values,dtype=torch.float)

In [11]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x_new,y,test_size=0.3,random_state=2)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

torch.Size([8492866, 6])
torch.Size([3639800, 6])
torch.Size([8492866])
torch.Size([3639800])


In [12]:
class carprice(nn.Module) :
    def __init__(self):
        super(carprice,self).__init__()
        self.layer1 = nn.Linear(6,64)
        self.layer2 = nn.Linear(64,128)
        self.layer3 = nn.Linear(128,64)
        self.layer4 = nn.Linear(64,6)
        self.layer5 = nn.Linear(6,1)
        self.relu  = nn.ReLU()
    def forward(self,x) :
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.relu(x)
        x = self.layer3(x)
        x = self.relu(x)
        x = self.layer4(x)
        x = self.relu(x)
        x = self.layer5(x)
        return x 

In [13]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))



True
NVIDIA GeForce RTX 3060 Laptop GPU


In [14]:
import torch

model = carprice()
# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Move model to GPU
model = model.to(device)

x_train = x_train.to(device)
y_train = y_train.to(device)
x_test  = x_test.to(device)
y_test  = y_test.to(device)


Using device: cuda


In [15]:
torch.cuda.empty_cache()

In [16]:
from torch.utils.data import DataLoader,TensorDataset
batch_size=128

train_dataset = TensorDataset(x_train,y_train)

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True
)
val_dataset = TensorDataset(x_test,y_test)
val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=True
)

In [17]:
import torch.optim as optim
from tqdm import tqdm
from sklearn.metrics import r2_score
loss = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

no_epochs = 10
train_preds = [] 
val_preds = [] 
train_target = [] 
val_target = [] 
for epoch in range(no_epochs) :
    train_loss = 0.0

    for x_batch,y_batch in tqdm(train_loader) : 
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        predicition = model(x_batch).squeeze(1)
        MSE = loss(predicition,y_batch)
        MSE.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss+=MSE.item()
        train_preds.append(predicition.detach().cpu())
        train_target.append(y_batch.detach().cpu())
    train_loss/=len(train_loader)
    train_rmse = train_loss**0.5
    model.eval()
    val_loss =0.0
  
    with torch.no_grad() :
        for x_batch,y_batch in tqdm(val_loader) :
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            predicition = model(x_batch).squeeze(1)
            MSE = loss(predicition,y_batch)
            val_loss+=MSE.item()
            val_preds.append(predicition.detach().cpu())
            val_target.append(y_batch.detach().cpu())
    train_pred_np = torch.cat(train_preds).numpy()
    train_target_np = torch.cat(train_target).numpy()
    val_preds_np = torch.cat(val_preds).numpy()
    val_target_np = torch.cat(val_target).numpy()
    train_r2 = r2_score(train_target_np,train_pred_np)
    val_r2 = r2_score(val_target_np,val_preds_np)
    val_loss /=len(val_loader)
    val_rmse = val_loss**0.5
    if (epoch + 1) % 1 == 0:
        print(
            f"Epoch [{epoch+1}/{no_epochs}] "
            f"Train Loss: {train_loss:.4f}, Train RMSE: {train_rmse:.4f}% | "
            f"Val Loss: {val_loss:.4f}, Val RMSE: {val_rmse:.4f}% |"
            f"train Accuracy : {train_r2:.4f}, Val Accuracy : {val_r2:.4f}"
        )

100%|██████████| 28436/28436 [00:32<00:00, 869.71it/s]


Epoch [1/10] Train Loss: 0.2354, Train RMSE: 0.4852% | Val Loss: 0.2267, Val RMSE: 0.4761% |train Accuracy : 0.7644, Val Accuracy : 0.7737


100%|██████████| 28436/28436 [00:32<00:00, 869.56it/s]


Epoch [2/10] Train Loss: 0.2263, Train RMSE: 0.4758% | Val Loss: 0.2263, Val RMSE: 0.4757% |train Accuracy : 0.7690, Val Accuracy : 0.7739


100%|██████████| 28436/28436 [00:32<00:00, 864.27it/s]


...

Epoch [8/10] Train Loss: 0.2257, Train RMSE: 0.4751% | Val Loss: 0.2254, Val RMSE: 0.4748% |train Accuracy : 0.7726, Val Accuracy : 0.7744


100%|██████████| 28436/28436 [00:32<00:00, 870.36it/s]


Epoch [9/10] Train Loss: 0.2256, Train RMSE: 0.4750% | Val Loss: 0.2255, Val RMSE: 0.4749% |train Accuracy : 0.7728, Val Accuracy : 0.7745


100%|██████████| 28436/28436 [00:32<00:00, 868.01it/s]


Epoch [10/10] Train Loss: 0.2256, Train RMSE: 0.4750% | Val Loss: 0.2258, Val RMSE: 0.4752% |train Accuracy : 0.7729, Val Accuracy : 0.7745
